<a href="https://colab.research.google.com/github/mcdodik20008/tg-bot/blob/master/notebooks/resume_kasphitskii_parashchuk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!wget https://raw.githubusercontent.com/mcdodik20008/tg-bot/refs/heads/master/datasets/combined/combined_df.csv

--2025-10-28 14:52:06--  https://raw.githubusercontent.com/mcdodik20008/tg-bot/refs/heads/master/datasets/combined/combined_df.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11490325 (11M) [text/plain]
Saving to: ‘combined_df.csv.1’

combined_df.csv.1   100%[===================>]  10.96M  --.-KB/s    in 0.03s   

2025-10-28 14:52:06 (349 MB/s) - ‘combined_df.csv.1’ saved [11490325/11490325]



In [26]:
import pandas as pd
import numpy as np
import re
import math

In [27]:
df = pd.read_csv('combined_df.csv')

In [28]:
df.head()

,full_name,desired_position,salary,personal_info,location,employment_and_schedule,total_experience,last_job,responsibilities,key_skills,about_me,education,languages,link
0,Кандидат,C# разработчик,35 000 ₽ на руки,"Мужчина, 31 год","Тверь , готов к переезду , готов к командировкам",Не указано,Опыт работы 4 года 11 месяцев,C# разработчик в Pet-projects,"Разработка WPF приложений, для получения опыта","Visual Studio C#, .NET Framework, Git, ООП, WP...",В последние годы работал в других специальност...,"ТПК им.Ф.В.Бадюлина Спортивный , Преподаватель...","Русский — Родной, Английский — A1 — Начальный",https://hh.ru/resume/844843fd000909db410039ed1...
1,Кандидат,Программист С#,70 000 ₽ на руки,"Мужчина, 37 лет","Мценск , готов к переезду (Орел) , не готов к ...",Не указано,Опыт работы 7 лет 9 месяцев,C#- разработчик в Фриланс,Разработка различного рода программ и приложен...,"Git, JavaScript, React, C#, .NET Framework, MS...",Профессионал своего дела! Изучил C# сам с нуля...,"МФ ФГБОУ ВО ""ОГУ имени И.С. Тургенева Экономик...","Русский — Родной, Французский — A1 — Начальный",https://hh.ru/resume/d683dbf4000858e0e00039ed1...
2,Кандидат,Backend разработчик .Net C#,350 000 ₽ на руки,"Мужчина, 46 лет","Москва , готов к переезду , готов к командировкам",Не указано,Опыт работы 24 года 9 месяцев,Разработчик C# в SDI Research,"Внедрение MDM-системы, разработка интеграционн...","C#, MS SQL Server, REST, Git, Entity Framework...",Женат. Двое детей. Хобби - музыка Реализованны...,Московский авиационный институт (национальный ...,"Русский — Родной, Английский — B1 — Средний",https://hh.ru/resume/735d51b700040ead140039ed1...
3,Кандидат,C# разработчик,Не указано,"Мужчина, Не указано","Санкт-Петербург , не готов к переезду , готов ...",Не указано,Опыт работы 1 год 11 месяцев,Fullstack-разработчик в ООО ABC,Веб-система управления инженерными данными (LI...,"C#, ASP. NET, .NET Core, Entity Framework, Раз...",В январе 2023 начал углубленно изучать основы ...,Санкт-Петербургский государственный университе...,Русский — Родной,https://hh.ru/resume/cb8d2bb300079af6910039ed1...
4,Кандидат,C# developer/разработчик,Не указано,"Мужчина, 27 лет","Санкт-Петербург , готов к переезду , готов к к...",Не указано,Опыт работы 2 года 11 месяцев,C# разработчик в Saber Interactive,Работа в Tools-отделе: - Поддержка внутренней ...,"Работоспособность, C#, Git, SQL, .NET Framewor...",В личное время работал над различными проектам...,Санкт-Петербургский национальный исследователь...,"Русский — Родной, Английский — B2 — Средне-про...",https://hh.ru/resume/e8fe8a530007fe538e0039ed1...


In [29]:
# Дропнули имя, не несет смысла
df.drop(columns=['full_name'], inplace=True)

In [30]:
# Дропнули занятость, не заполнено практически нигде
df.drop(columns=['employment_and_schedule'], inplace=True)

In [31]:
# Разделили последнее место работы
df[['last_position', 'last_job_company']] = df['last_job'].str.split(' в ', n=1, expand=True)

df.drop(columns=['last_job'], inplace=True)

In [32]:
# Разделили пол и возраст
df[['gender', 'age']] = df['personal_info'].str.split(', ', n=1, expand=True)
df.drop(columns=['personal_info'], inplace=True)


df['age'] = df['age'].str.extract('(\\d+)').astype('Int64')

In [33]:
def parse_experience(exp_str):
    if pd.isna(exp_str):
        return pd.NA

    text = str(exp_str).replace('Опыт работы', '').strip()

    years, months = 0, 0

    year_match = re.search(r'(\d+)\s*(?:год|года|лет)', text)
    if year_match:
        years = int(year_match.group(1))

    month_match = re.search(r'(\d+)\s*(?:месяц|месяца|месяцев)', text)
    if month_match:
        months = int(month_match.group(1))

    if months >= 6:
        years += 1

    return years

# Привели к числам опыт
df['experience_years'] = df['total_experience'].apply(parse_experience).astype('Int64')
df.drop(columns=['total_experience'], inplace=True)

In [34]:
# Обработка зарплаты
def parse_salary(salary_str):
    if pd.isna(salary_str):
        return np.nan, np.nan

    numbers = re.findall(r'\d+', str(salary_str))
    if not numbers:
        return np.nan, np.nan

    amount = int("".join(numbers))

    salary_lower = str(salary_str).lower()
    currency = 'RUB'
    if '$' in salary_lower or 'usd' in salary_lower:
        currency = 'USD'
    elif '€' in salary_lower or 'eur' in salary_lower:
        currency = 'EUR'
    elif '₸' in salary_lower or 'kzt' in salary_lower:
        currency = 'KZT'

    return amount, currency

exchange_rates = { 'RUB': 1, 'USD': 95, 'EUR': 102, 'KZT': 0.2 }

df[['salary_amount', 'currency']] = df['salary'].apply(lambda x: pd.Series(parse_salary(x)))
df['salary_rub'] = df.apply(
    lambda row: row['salary_amount'] * exchange_rates.get(row['currency'], 0) if pd.notna(row['salary_amount']) else np.nan,
    axis=1
).astype('Int64')
df.drop(columns=['salary_amount', 'currency', 'salary'], inplace=True)

In [35]:
# Обработка местоположения
def parse_location(location_str):
    if pd.isna(location_str):
        return np.nan, np.nan, np.nan

    parts = [p.strip() for p in location_str.split(',')]
    city = parts[0]
    can_move = np.nan
    can_trip = np.nan

    for part in parts[1:]:
        if 'переезду' in part:
            can_move = 'не готов' not in part
        if 'командировкам' in part:
            can_trip = 'не готов' not in part

    return city, can_move, can_trip

df[['city', 'can_move', 'can_trip']] = df['location'].apply(lambda x: pd.Series(parse_location(x)))

df['can_move'] = df['can_move'].astype('boolean')
df['can_trip'] = df['can_trip'].astype('boolean')
df.drop(columns=['location'], inplace=True)

In [36]:
df['gender'] = df['gender'].apply(lambda x: 0 if x == 'Мужчина' else (1 if x == 'Женщина' else -1))

In [37]:
df.head()

,desired_position,responsibilities,key_skills,about_me,education,languages,link,last_position,last_job_company,gender,age,experience_years,salary_rub,city,can_move,can_trip
0,C# разработчик,"Разработка WPF приложений, для получения опыта","Visual Studio C#, .NET Framework, Git, ООП, WP...",В последние годы работал в других специальност...,"ТПК им.Ф.В.Бадюлина Спортивный , Преподаватель...","Русский — Родной, Английский — A1 — Начальный",https://hh.ru/resume/844843fd000909db410039ed1...,C# разработчик,Pet-projects,0,31,5,35000,Тверь,True,True
1,Программист С#,Разработка различного рода программ и приложен...,"Git, JavaScript, React, C#, .NET Framework, MS...",Профессионал своего дела! Изучил C# сам с нуля...,"МФ ФГБОУ ВО ""ОГУ имени И.С. Тургенева Экономик...","Русский — Родной, Французский — A1 — Начальный",https://hh.ru/resume/d683dbf4000858e0e00039ed1...,C#- разработчик,Фриланс,0,37,8,70000,Мценск,True,False
2,Backend разработчик .Net C#,"Внедрение MDM-системы, разработка интеграционн...","C#, MS SQL Server, REST, Git, Entity Framework...",Женат. Двое детей. Хобби - музыка Реализованны...,Московский авиационный институт (национальный ...,"Русский — Родной, Английский — B1 — Средний",https://hh.ru/resume/735d51b700040ead140039ed1...,Разработчик C#,SDI Research,0,46,25,350000,Москва,True,True
3,C# разработчик,Веб-система управления инженерными данными (LI...,"C#, ASP. NET, .NET Core, Entity Framework, Раз...",В январе 2023 начал углубленно изучать основы ...,Санкт-Петербургский государственный университе...,Русский — Родной,https://hh.ru/resume/cb8d2bb300079af6910039ed1...,Fullstack-разработчик,ООО ABC,0,<NA>,2,<NA>,Санкт-Петербург,False,True
4,C# developer/разработчик,Работа в Tools-отделе: - Поддержка внутренней ...,"Работоспособность, C#, Git, SQL, .NET Framewor...",В личное время работал над различными проектам...,Санкт-Петербургский национальный исследователь...,"Русский — Родной, Английский — B2 — Средне-про...",https://hh.ru/resume/e8fe8a530007fe538e0039ed1...,C# разработчик,Saber Interactive,0,27,3,<NA>,Санкт-Петербург,True,True


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5109 entries, 0 to 5108
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   desired_position  5109 non-null   object 
 1   responsibilities  5109 non-null   object 
 2   key_skills        5109 non-null   object 
 3   about_me          5109 non-null   object 
 4   education         5109 non-null   object 
 5   languages         5109 non-null   object 
 6   link              5109 non-null   object 
 7   last_position     5109 non-null   object 
 8   last_job_company  4741 non-null   object 
 9   gender            5109 non-null   int64  
 10  age               4886 non-null   Int64  
 11  experience_years  5109 non-null   Int64  
 12  salary_rub        3074 non-null   Int64  
 13  city              5109 non-null   object 
 14  can_move          4907 non-null   boolean
 15  can_trip          5005 non-null   boolean
dtypes: Int64(3), boolean(2), int64(1), object(

In [39]:
non_it_keywords = [
    'не указано', 'водитель', 'бухгалтер', 'охранник', 'продавец', 'юрист', 'экономист',
    'секретарь', 'кассир', 'дизайнер', 'трейдер', 'ретушер', 'переводчик',
    'журналист', 'эколог', 'менеджер по продажам', 'smm', 'офис-менеджер'
]

# Отсеиваем не IT опыт
df = df[~df['last_position'].str.lower().apply(lambda x: any(keyword in x for keyword in non_it_keywords))].copy()

df.reset_index(drop=True, inplace=True)

In [40]:
non_it_keywords = [
    'не указано', 'водитель', 'бухгалтер', 'охранник', 'продавец', 'юрист', 'экономист',
    'секретарь', 'кассир', 'дизайнер', 'трейдер', 'ретушер', 'переводчик',
    'журналист', 'эколог', 'менеджер по продажам', 'smm', 'офис-менеджер', 'cgi',
    'defi', 'спепиалист', 'репетитор', 'хостес', 'ремонт', 'налог', 'техник', 'математик', 'гипсокартон'
]

# Отсеиваем не IT должности
df = df[~df['desired_position'].str.lower().apply(lambda x: any(keyword in x for keyword in non_it_keywords))].copy()

df.reset_index(drop=True, inplace=True)
df.head()

,desired_position,responsibilities,key_skills,about_me,education,languages,link,last_position,last_job_company,gender,age,experience_years,salary_rub,city,can_move,can_trip
0,C# разработчик,"Разработка WPF приложений, для получения опыта","Visual Studio C#, .NET Framework, Git, ООП, WP...",В последние годы работал в других специальност...,"ТПК им.Ф.В.Бадюлина Спортивный , Преподаватель...","Русский — Родной, Английский — A1 — Начальный",https://hh.ru/resume/844843fd000909db410039ed1...,C# разработчик,Pet-projects,0,31,5,35000,Тверь,True,True
1,Программист С#,Разработка различного рода программ и приложен...,"Git, JavaScript, React, C#, .NET Framework, MS...",Профессионал своего дела! Изучил C# сам с нуля...,"МФ ФГБОУ ВО ""ОГУ имени И.С. Тургенева Экономик...","Русский — Родной, Французский — A1 — Начальный",https://hh.ru/resume/d683dbf4000858e0e00039ed1...,C#- разработчик,Фриланс,0,37,8,70000,Мценск,True,False
2,Backend разработчик .Net C#,"Внедрение MDM-системы, разработка интеграционн...","C#, MS SQL Server, REST, Git, Entity Framework...",Женат. Двое детей. Хобби - музыка Реализованны...,Московский авиационный институт (национальный ...,"Русский — Родной, Английский — B1 — Средний",https://hh.ru/resume/735d51b700040ead140039ed1...,Разработчик C#,SDI Research,0,46,25,350000,Москва,True,True
3,C# разработчик,Веб-система управления инженерными данными (LI...,"C#, ASP. NET, .NET Core, Entity Framework, Раз...",В январе 2023 начал углубленно изучать основы ...,Санкт-Петербургский государственный университе...,Русский — Родной,https://hh.ru/resume/cb8d2bb300079af6910039ed1...,Fullstack-разработчик,ООО ABC,0,<NA>,2,<NA>,Санкт-Петербург,False,True
4,C# developer/разработчик,Работа в Tools-отделе: - Поддержка внутренней ...,"Работоспособность, C#, Git, SQL, .NET Framewor...",В личное время работал над различными проектам...,Санкт-Петербургский национальный исследователь...,"Русский — Родной, Английский — B2 — Средне-про...",https://hh.ru/resume/e8fe8a530007fe538e0039ed1...,C# разработчик,Saber Interactive,0,27,3,<NA>,Санкт-Петербург,True,True


In [41]:
!pip install pymorphy3

In [42]:
import pandas as pd
import re
import pymorphy3

morph = pymorphy3.MorphAnalyzer()

# 3. Функция предобработки (Лемматизация + Очистка)
def preprocess_title(text):
    if not isinstance(text, str):
        return ''

    text_lower = text.lower()

    # Заменяем все разделители на пробелы ДО очистки.
    # 'web-разработчик' -> 'web разработчик'
    # 'c++/python' -> 'c++ python'
    text_spaced = re.sub(r'[-/\\,]', ' ', text_lower)

    # Очищаем русские слова для лемматизатора
    # (теперь 'web разработчик' -> '   разработчик')
    # (а 'программист с#' -> 'программист с ')
    text_rus_only = re.sub(r'[^а-я\s]', ' ', text_spaced)

    # Лемматизируем
    lemmatized_words = [morph.parse(w)[0].normal_form for w in text_rus_only.split()]
    lemmatized_string = ' '.join(lemmatized_words)

    # Финальная строка для поиска:
    # text_spaced - содержит 'web', 'c#', '.net'
    # lemmatized_string - содержит 'программист', 'разработчик'
    return text_spaced + ' ' + lemmatized_string

# 4. Применяем предобработку
df['job_title_search'] = df['desired_position'].apply(preprocess_title)

# 5. Функция-категоризатор
# 5. ФИНАЛЬНАЯ ВЕРСИЯ. ДОПИЛЕННАЯ.
def categorize_job_regex(title):

    # --- Блок 1: Non-IT ---
    # (Без изменений)
    non_it_pattern = r'\b(водитель|бухгалтер|охранник|продавец|юрист|экономист|секретарь|кассир|дизайнер|трейдер|ретушер|переводчик|журналист|эколог|маркетинг|smm|офис|контент|персонал|закупка|оператор|консультант|юрист|кассир|торговый|представитель|охранник|кладовщик|администратор|помощник|агент|монтажник|слесарь|химик|механик|электрик|электромонтёр|сварщик)\b'
    if re.search(non_it_pattern, title):
        if re.search(r'администратор', title) and re.search(r'\b(системный|сеть|база|бд|it|ит|иб|linux|windows|сисадмин)\b', title):
            pass
        else:
            return 'Non-IT'

    # --- Блок 2: Конкретные Технологии и Роли ---

    if re.search(r'data scientist|data science|\bds\b|\bml\b|machine learning|машинный обучение|data engineer|deep|аналитик данные|bi|computer vision', title):
        return 'Data Science / Data'
    if re.search(r'\b1c\b|\b1с\b', title):
        return '1C'
    if re.search(r'\bqa\b|\btest\b|тестировщик|тестирование|aqa', title):
        return 'QA'
    if re.search(r'devops', title):
        return 'DevOps'
    if re.search(r'security|безопасность|\bиб\b', title):
        return 'InfoSec'

    # !!! НОВАЯ КАТЕГОРИЯ !!! (Тех. поддержка)
    if re.search(r'support|поддержка|helpdesk', title):
        return 'Support / HelpDesk'

    if re.search(r'android|ios|mobile|мобильный|flutter|react native|kotlin|swift|objective-c|xamarin', title):
        return 'Mobile'
    if re.search(r'fullstack|full stack|фуллстек', title):
        return 'Fullstack'

    # (Frontend фреймворки - без изменений)
    if re.search(r'\bangular\b', title):
        return 'Frontend (Angular)'
    if re.search(r'\breact\b', title):
        return 'Frontend (React)'
    if re.search(r'\bvue\b', title):
        return 'Frontend (Vue)'

    # !!! ФИКС ДЛЯ Node.js !!! (ловим 'nodejs' слитно)
    if re.search(r'node\.?js|\bnode\b', title):
        return 'Backend (Node.js)'

    # !!! ФИКС ДЛЯ FRONTEND !!! (ловим 'фронтэнд' и 'fe')
    if re.search(r'\bweb\b|\bвеб\b|front[\s\-]?end|фронтенд|фронтэнд|font[\s\-]?end|frontent|верстальщик|html|css|\bui\b|javascript|\bjs\b|\bfe\b', title):
        return 'Frontend'

    # Python
    if re.search(r'\bpython\b|\bпитон\b', title):
        return 'Python'

    # C# / .NET
    if re.search(r'[cс]\#|\.net', title):
        return 'C# / .NET'

    # Java
    if re.search(r'\bjava\b(?!\s*script)', title):
        return 'Java'

    # Go
    if re.search(r'\bgo\b|\bgolang\b', title):
        return 'Go'

    # C++
    if re.search(r'c\+\+|с\+\+', title):
        return 'C++'

    # PHP
    if re.search(r'\bphp\b', title):
        return 'PHP'

    # Ruby
    if re.search(r'\bruby\b', title):
        return 'Ruby'

    # !!! НОВЫЕ ЯЗЫКИ !!!
    if re.search(r'\brust\b', title):
        return 'Rust'
    if re.search(r'\bclojure\b', title):
        return 'Clojure'
    if re.search(r'\bpascal\b', title):
        return 'Pascal'

    # !!! НОВЫЙ ЯЗЫК 'C' (ВАЖНО: после C++ и C#) !!!
    if re.search(r'\b[cс]\b', title):
        return 'C'

    # DBA / SQL
    if re.search(r'sql|pl/sql|администратор бд|\bбд\b|база данные|oracle|postgres', title):
        return 'DBA / SQL'

    # !!! ФИКС ДЛЯ GAMEDEV !!! (ловим 'unity3d' и т.д.)
    if re.search(r'\bunity|game|\bигра\b|unreal engine', title):
        return 'GameDev'

    # !!! ФИКС ДЛЯ SAP !!! (убираем \b)
    if re.search(r'\bsap|\babap\b', title):
        return 'SAP / ABAP'

    # !!! НОВЫЕ ПЛАТФОРМЫ/ТЕХНОЛОГИИ !!!
    if re.search(r'\bdelphi\b', title):
        return 'Delphi'
    if re.search(r'\bbitrix\b|битрикс', title):
        return 'Bitrix'
    if re.search(r'sharepoint', title):
        return 'SharePoint'
    if re.search(r'\blotus\b', title):
        return 'Lotus'
    if re.search(r'\brpa\b', title):
        return 'RPA'
    if re.search(r'\bflash\b', title):
        return 'Flash'

    # !!! НОВЫЕ ИНЖЕНЕРНЫЕ КАТЕГОРИИ !!!
    if re.search(r'асу|кипиа|плк|plc', title): # АСУ ТП / КИПиА
        return 'АСУ ТП / КИПиА'
    if re.search(r'embedded|встраиваемый|микроконтроллер|микропроцессорный|dsp|qnx|rtos', title): # Embedded
        return 'Embedded'
    if re.search(r'fpga|плис|рэа|схемотехник|электронщик', title): # Hardware
        return 'Hardware / FPGA'


    # --- Блок 3: Общие Роли (когда стек не ясен) ---

    if re.search(r'manager|lead|\b(руководитель|менеджер|teamlead|тимлид|cto|директор|project|архитектор)\b', title):
        return 'Manager / Lead'
    if re.search(r'\b(аналитик|analyst)\b', title):
        return 'System Analyst'
    if re.search(r'системный администратор|\bsysadmin\b|\bсисадмин\b|администратор сеть|linux|windows', title):
        return 'SysAdmin'

    # !!! ФИКС ДЛЯ BACKEND !!! (ловим 'back end' с пробелом)
    if re.search(r'back[\s\-]?end|бэкенд|api|django', title):
        return 'Backend (Other)'

    # --- Блок 4: Все остальные IT-шники ---
    # (Это наша "корзина" для 'Программист', 'Инженер' и т.д.)
    if re.search(r'\b(программист|разработчик|developer|engineer|инженер|it|ит|software|специалист по)\b', title):
        return 'Other IT'

    # --- Блок 5: Все остальное, что не опознали ---
    return 'Other (Unknown)'

# 6. Применяем новую функцию
df['category'] = df['job_title_search'].apply(categorize_job_regex)

# 7. СМОТРИМ РЕЗУЛЬТАТ
print("------ Итоговые категории и их количество: ------")
print(df['category'].value_counts())

# 8. Проверяем "неизвестных"
print("\n\n------ Что попало в 'Other (Unknown)': ------")
unknown_jobs = df[df['category'] == 'Other (Unknown)']['desired_position'].unique()

print(list(unknown_jobs))

------ Итоговые категории и их количество: ------
category
Other IT               2219
Frontend                702
C# / .NET               239
Fullstack               118
SysAdmin                117
1C                      105
Python                  103
Java                    102
PHP                      85
Frontend (React)         75
Backend (Other)          72
C++                      70
Non-IT                   55
DevOps                   45
Mobile                   44
Frontend (Angular)       37
Other (Unknown)          34
Go                       29
Frontend (Vue)           24
АСУ ТП / КИПиА           21
Manager / Lead           21
Data Science / Data      18
Embedded                 18
DBA / SQL                18
Backend (Node.js)        17
Support / HelpDesk       17
Delphi                   15
GameDev                  14
System Analyst           13
C                        12
Hardware / FPGA          11
Ruby                      7
QA                        6
InfoSec          

In [43]:
other_it = df[df['category'] == 'Other IT']['desired_position'].unique()
other_it

array(['Junior программист', 'Программист', 'Программист-разработчик',
       'Программист, разработчик', 'Разработчик',
       'Разработчик-программист', 'Ведущий разработчик',
       'Software Developer', 'Senior Software Engineer',
       'Главный инженер', 'Инженер-электроник', 'Тимлидер/Разработчик',
       'Software Engineer', 'Инженер', 'Инженер-программист', 'Developer',
       'Senior Developer (Ведущий разработчик) / CIO / VP of Engineering',
       'APPLICATION DEVELOPER',
       'Product Designer / Design System Engineer', 'программист',
       'инженер-программист', 'Программист инженер',
       'Инженер - программист',
       'Ведущий инженер, Инженер-программист, Инженер-технолог',
       'Инженер- программист',
       'Инженер-Программист, Программист-разработчик, Программист, Разработчик',
       'Инженер - Программист', 'Программист, Разработчик ПО',
       'Инженер, программист', 'IT-инженер', 'Инженер-разработчик',
       'Программист1', 'программист-разработчик', '

In [44]:
df.drop(columns=['job_title_search'], inplace=True)
df.reset_index(drop=True, inplace=True)

In [45]:
df.sample()

,desired_position,responsibilities,key_skills,about_me,education,languages,link,last_position,last_job_company,gender,age,experience_years,salary_rub,city,can_move,can_trip,category
1738,DevOps инженер,Создание CI/CD на платформе Azure. Поддержание...,"Jenkins, Администрирование сайтов, Техническая...",Обладаю навыками работы с технологией Ci/CD. В...,Санкт-Петербургский национальный исследователь...,"Русский — Родной, Английский — B1 — Средний",https://hh.ru/resume/f798b71f0002e27a4c0039ed1...,Инженер-программист,Инсайрес,0,33,6,150000,Волгоград,True,False,DevOps


In [46]:
# 1. Увеличиваем лимит отображения, чтобы увидеть все 200
pd.set_option('display.max_rows', 200)

# 2. Разворачиваем все скиллы в один столб
all_skills = df['key_skills'].copy().dropna().str.split(r'[,/|]').explode()

# 3. Чистим и считаем
top_skills = all_skills.str.strip().str.lower().value_counts()

# 4. Выводим топ-200
print("------ Топ 200 самых частых скиллов ------")
print(top_skills.head(200))

------ Топ 200 самых частых скиллов ------
key_skills
русский — родной                           4303
git                                        1542
javascript                                 1454
английский — b2 — средне-продвинутый       1358
sql                                        1097
html                                       1091
английский — a1 — начальный                1084
css                                         939
mysql                                       874
html5                                       811
linux                                       784
c++                                         708
php                                         693
ооп                                         692
css3                                        671
postgresql                                  661
jquery                                      638
английский — b1 — средний                   594
c#                                          590
python                            

In [47]:
# Проведем категоризацию по скилам

# --- Шаг 1: Подготовка колонки 'key_skills' ---
def preprocess_skills(skills):
    if not isinstance(skills, str):
        return ''
    skills_lower = skills.lower()
    # "JavaScript, CSS/HTML" -> "javascript css html"
    skills_spaced = re.sub(r'[,/\\|]', ' ', skills_lower)
    return skills_spaced

df['skills_search'] = df['key_skills'].apply(preprocess_skills)


def categorize_by_skills(skills_str):

    # --- Блок 1: Платформы (CMS, SAP, 1C) ---

    # PHP CMS (до 1С)
    if re.search(r'\bbitrix\b|битрикс|wordpress|joomla', skills_str):
        return 'PHP'

    # 1C
    if re.search(r'\b1c\b|\b1с\b', skills_str):
        return '1C'

    # SAP
    if re.search(r'\bsap\b|\babap\b', skills_str):
        return 'SAP / ABAP'

    # SharePoint
    if re.search(r'sharepoint', skills_str):
        return 'SharePoint'

    # --- Блок 2: Data Science / ML ---
    # (До Python)
    if re.search(r'data scientist|machine learning|keras|pandas|numpy|scikit|tensorflow|pytorch|\bmatlab\b', skills_str):
        return 'Data Science / Data'

    # --- Блок 3: DevOps / SysAdmin ---
    # (До Linux)
    if re.search(r'devops|docker|kubernetes|jenkins|ansible|terraform|\bci\b|\bcd\b', skills_str):
        return 'DevOps'

    if re.search(r'sysadmin|active directory|windows server|администрирование|настройка сервер', skills_str):
        return 'SysAdmin'

    # --- Блок 4: QA ---
    if re.search(r'\bqa\b|selenium|pytest|junit|cypress|playwright|тестирование', skills_str):
        return 'QA'

    # --- Блок 5: Mobile ---
    if re.search(r'android|ios|mobile|flutter|react native|kotlin|swift|objective-c|xamarin', skills_str):
        return 'Mobile'

    # --- Блок 6: Frontend (Фреймворки) ---
    # (До общего JS)
    if re.search(r'\bangular\b', skills_str):
        return 'Frontend (Angular)'
    if re.search(r'\breact\b|\bredux\b|\bnext\.js\b|\bmobx\b|\bjest\b', skills_str):
        return 'Frontend (React)'
    if re.search(r'\bvue\b|\bvuex\b', skills_str):
        return 'Frontend (Vue)'

    # --- Блок 7: Backend (Node.js) ---
    # (До общего JS)
    if re.search(r'node\.?js|\bnode\b', skills_str):
        return 'Backend (Node.js)'

    # --- Блок 8: Frontend (Общий) ---
    if re.search(r'frontend|\bjs\b|javascript|html|css|jquery|bootstrap|typescript|sass|scss|less|webpack|gulp|бэм|верстка', skills_str):
        # Проверка, что это не Java
        if re.search(r'\bjava\b(?!\s*script)', skills_str):
             pass
        else:
             return 'Frontend'

    # --- Блок 9: Backend (Языки) ---
    if re.search(r'\bpython\b|\bdjango\b|\bflask\b|\bfastapi\b|\bcelery\b', skills_str):
        return 'Python'
    if re.search(r'[cс]\#|\.net|\basp\b|entity framework|\bwpf\b|\blinq\b|windows forms', skills_str):
        return 'C# / .NET'
    if re.search(r'\bjava\b(?!\s*script)|\bspring\b|\bhibernate\b|\bmaven\b', skills_str):
        return 'Java'
    if re.search(r'\bphp\b|\blaravel\b|\byii\b', skills_str):
        return 'PHP'
    if re.search(r'\bgo\b|\bgolang\b', skills_str):
        return 'Go'
    if re.search(r'c\+\+|с\+\+|\bqt\b|\bstl\b', skills_str):
        return 'C++'
    if re.search(r'\bruby\b|\brails\b', skills_str):
        return 'Ruby'

    # --- Блок 10: GameDev / Delphi / Pascal ---
    if re.search(r'\bunity\b', skills_str):
        return 'GameDev'
    if re.search(r'\bdelphi\b|object pascal', skills_str):
        return 'Delphi'
    if re.search(r'\bpascal\b', skills_str):
        return 'Pascal'

    # --- Блок 11: Low-level / C ---
    # (После C++ и C#)
    if re.search(r'\bassembler\b', skills_str):
        return 'Embedded'
    if re.search(r'\b[cс]\b', skills_str):
        return 'C'

    # --- Блок 12: Другие IT-роли (низкий приоритет) ---
    if re.search(r'техническая поддержка|support', skills_str):
        return 'Support / HelpDesk'

    # Linux (низкий приоритет, т.к. многие его знают)
    if re.search(r'\blinux\b', skills_str):
        return 'SysAdmin'

    # SQL (самый низкий приоритет, т.к. ВСЕ его знают)
    if re.search(r'sql|mysql|postgresql|mssql|oracle', skills_str):
        return 'DBA / SQL'

    # --- Блок 13: Non-IT (Дизайн и т.п.) ---
    if re.search(r'photoshop|figma|coreldraw|autocad', skills_str):
        return 'Non-IT (Design)'

    # Если не нашли НИЧЕГО из этого - не меняем категорию
    return None

# --- Шаг 3: Применение логики (Без изменений) ---

print("------ СТАТИСТИКА КАТЕГОРИЙ (ДО УТОЧНЕНИЯ): ------")
print(df['category'].value_counts())

# 1. Маска для 'Other IT'
mask_other_it = df['category'] == 'Other IT'

# 2. Применяем функцию по скиллам
new_cats_from_skills = df.loc[mask_other_it, 'skills_search'].apply(categorize_by_skills)

# 3. Обновляем 'category'
df.loc[mask_other_it & new_cats_from_skills.notna(), 'category'] = new_cats_from_skills[new_cats_from_skills.notna()]


print("\n\n------ СТАТИСТИКА КАТЕГОРИЙ (ПОСЛЕ УТОЧНЕНИЯ): ------")
# Ожидаем, что 'Other IT' сильно уменьшится
print(df['category'].value_counts())

print("\n\n------ Что ОСТАЛОСЬ в 'Other IT': ------")
# Тут должны быть 'Программисты' со скиллами типа 'Git, ООП, Agile, HTTP'
remaining_other_it = df[df['category'] == 'Other IT']
print(remaining_other_it[['desired_position', 'key_skills']].head(20))

------ СТАТИСТИКА КАТЕГОРИЙ (ДО УТОЧНЕНИЯ): ------
category
Other IT               2219
Frontend                702
C# / .NET               239
Fullstack               118
SysAdmin                117
1C                      105
Python                  103
Java                    102
PHP                      85
Frontend (React)         75
Backend (Other)          72
C++                      70
Non-IT                   55
DevOps                   45
Mobile                   44
Frontend (Angular)       37
Other (Unknown)          34
Go                       29
Frontend (Vue)           24
АСУ ТП / КИПиА           21
Manager / Lead           21
Data Science / Data      18
Embedded                 18
DBA / SQL                18
Backend (Node.js)        17
Support / HelpDesk       17
Delphi                   15
GameDev                  14
System Analyst           13
C                        12
Hardware / FPGA          11
Ruby                      7
QA                        6
InfoSec         

In [48]:
# 1. Создаем словарь-маппинг
category_mapping = {
    # --- 1. Frontend ---
    'Frontend (React)': 'Frontend (Angular/React/Vue)',
    'Frontend (Angular)': 'Frontend (Angular/React/Vue)',
    'Frontend (Vue)': 'Frontend (Angular/React/Vue)',
    'Flash': 'Frontend',

    # --- 2. Backend (Niche) ---
    'Backend (Other)': 'Backend (Other)',
    'Backend (Node.js)': 'Backend (Other)',
    'Go': 'Backend (Other)',
    'Ruby': 'Backend (Other)',
    'Rust': 'Backend (Other)',
    'Clojure': 'Backend (Other)',

    # --- 3. Data & Analytics ---
    'Data Science / Data': 'Data & Analytics',
    'DBA / SQL': 'Data & Analytics',
    'System Analyst': 'Data & Analytics',

    # --- 4. Legacy / C ---
    'C': 'Legacy / C',
    'Delphi': 'Legacy / C',
    'Pascal': 'Legacy / C',

    # --- 5. Niche IT ---
    'QA': 'Niche IT',
    'Support / HelpDesk': 'Niche IT',
    'Embedded': 'Niche IT',
    'АСУ ТП / КИПиА': 'Niche IT',
    'SAP / ABAP': 'Niche IT',
    'Manager / Lead': 'Niche IT',
    'GameDev': 'Niche IT',
    'Hardware / FPGA': 'Niche IT',
    'SharePoint': 'Niche IT',
    'InfoSec': 'Niche IT',
    'RPA': 'Niche IT',
    'Lotus': 'Niche IT',

    # --- 6. Non-IT / Unknown ---
    'Non-IT': 'Non-IT / Unknown',
    'Other (Unknown)': 'Non-IT / Unknown',
    'Non-IT (Design)': 'Non-IT / Unknown',

    # --- 7. PHP ---
    'Bitrix': 'PHP'
}

# 2. Применяем маппинг
df['category'] = df['category'].replace(category_mapping)

# 3. СМОТРИМ ФИНАЛЬНЫЙ РЕЗУЛЬТАТ
print("------ СТАТИСТИКА КАТЕГОРИЙ: ------")
print(df['category'].value_counts())

------ СТАТИСТИКА КАТЕГОРИЙ: ------
category
Frontend                        908
Other IT                        819
C# / .NET                       333
1C                              322
SysAdmin                        274
PHP                             186
Python                          175
Frontend (Angular/React/Vue)    173
Niche IT                        172
Backend (Other)                 154
Java                            143
C++                             133
Data & Analytics                129
Legacy / C                      125
Fullstack                       118
Mobile                          118
DevOps                          114
Non-IT / Unknown                107
Name: count, dtype: int64
